# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import string
import re
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
# load data from database
engine = sqlite3.connect('DisasterResponse.db')
df = pd.read_sql("SELECT * FROM messages_disaster", con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x_cols = list(df.columns)[1]
y_cols = list(df.columns)[4:]

In [5]:
x_cols

'message'

In [6]:
X = df[x_cols]
Y = df[y_cols]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
len(Y.columns)

36

In [8]:
len(X)

26216

In [9]:
X_test.shape

(6554,)

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    '''
    Input:
        text (str): natural langueage text
    Output:
        clean_tokens (list): list of clean tokens
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    text.translate(str.maketrans('', '', string.punctuation))   
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
# from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [13]:
param_list = pipeline.get_params()
for k in param_list.keys():
    print(f'---------------{k}-----------------------')
    print(param_list[k])

---------------memory-----------------------
None
---------------steps-----------------------
[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f53ed084378>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=F

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
from sklearn.metrics import classification_report
def display_results(y_true, y_pred):
    target_names = list(y_true.columns)
    for i, col in enumerate(y_true):
        print(f'{col} |------------------------------>')
        print(classification_report(y_true[col], y_pred[:,i], target_names=target_names))

In [15]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [16]:
# display results
display_results(y_test, y_pred)


related |------------------------------>
                        precision    recall  f1-score   support

               related       0.64      0.36      0.46      1535
               request       0.82      0.94      0.87      4963
                 offer       0.71      0.09      0.16        56

           avg / total       0.77      0.79      0.77      6554

request |------------------------------>
                        precision    recall  f1-score   support

               related       0.89      0.98      0.93      5428
               request       0.82      0.39      0.53      1126

           avg / total       0.88      0.88      0.86      6554

offer |------------------------------>
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6527
               request       0.00      0.00      0.00        27

           avg / total       0.99      1.00      0.99      6554

aid_related |-------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# parameter searh is limited because of processing limitations
parameters = {'clf__estimator__n_estimators': [20, 30, 50] }
# 'vect__ngram_range': ((1, 1), (1, 2))
# 'vect__max_df': (0.5, 0.75, 1.0)


In [21]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [20, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
y_pred = cv.predict(X_test)

In [24]:
# display results
display_results(y_test, y_pred)

related |------------------------------>
                        precision    recall  f1-score   support

               related       0.74      0.29      0.42      1535
               request       0.81      0.97      0.88      4963
                 offer       1.00      0.09      0.16        56

           avg / total       0.79      0.80      0.77      6554

request |------------------------------>
                        precision    recall  f1-score   support

               related       0.89      0.99      0.94      5428
               request       0.87      0.42      0.57      1126

           avg / total       0.89      0.89      0.87      6554

offer |------------------------------>
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6527
               request       0.00      0.00      0.00        27

           avg / total       0.99      1.00      0.99      6554

aid_related |-------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
!pip install catboost

    100% |████████████████████████████████| 64.7MB 570kB/s eta 0:00:01  0% |▎                               | 512kB 9.6MB/s eta 0:00:07    4% |█▍                              | 2.9MB 22.6MB/s eta 0:00:03    16% |█████▏                          | 10.5MB 21.8MB/s eta 0:00:03    17% |█████▊                          | 11.6MB 24.4MB/s eta 0:00:03    21% |██████▉                         | 13.9MB 25.1MB/s eta 0:00:03    23% |███████▍                        | 15.0MB 22.8MB/s eta 0:00:03    24% |████████                        | 16.1MB 21.6MB/s eta 0:00:03    26% |████████▌                       | 17.1MB 23.2MB/s eta 0:00:03    28% |█████████                       | 18.2MB 22.6MB/s eta 0:00:03    29% |█████████▌                      | 19.3MB 24.3MB/s eta 0:00:02    32% |██████████▌                     | 21.3MB 22.5MB/s eta 0:00:02    36% |███████████▌                    | 23.4MB 21.7MB/s eta 0:00:02    39% |████████████▋                   | 25.4MB 21.7MB/s eta 0:00:02    43% |██████████████    

In [36]:
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
parameters = {'clf__estimator__leaf_size': [20, 30, 50] }
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
cv = GridSearchCV(pipeline, param_grid=parameters)

In [37]:
param_list = pipeline.get_params()
for k in param_list.keys():
    print(f'---------------{k}-----------------------')
    print(param_list[k])

---------------memory-----------------------
None
---------------steps-----------------------
[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f53ed084378>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]
---------------vect-----------------------
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
# display results
display_results(y_test, y_pred)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('trained_model.plk', 'wb+'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.